# Laboratorio 1: Regresión en California

En este laboratorio deben hacer experimentos de regresión con el conjunto de datos "California Housing dataset".

Estudiarán el dataset, harán visualizaciones y seleccionarán atributos relevantes a mano.

Luego, entrenarán y evaluarán diferentes tipos de regresiones, buscando las configuraciones que mejores resultados den.

In [ ]:
import numpy as np
import matplotlib.pyplot as plt

## Carga del Conjunto de Datos

Cargamos el conjunto de datos y vemos su contenido.

In [ ]:
from sklearn.datasets import fetch_california_housing
X_california, y_california = fetch_california_housing(return_X_y=True, as_frame=True)
california = fetch_california_housing()

In [ ]:
california.keys()

### Vistazo a la Estructura de Datos

In [ ]:

print(california['DESCR'])  # descripción del dataset
#california['feature_names'] # nombres de los atributos para cada columna de 'data'
#california['data']           # matriz con los datos de entrada (atributos)
#print(california['target']) # vector de valores a predecir

In [ ]:
california['data'].shape, california['target'].shape

Cada fila representa un distrito, o "block"

In [ ]:
X_california.head()

In [ ]:
y_california.head()

Información sobre los datos con `pd.DataFrame.info()` \
Hay 20640 registros, con 8 atributos, todo numéricos y ninguno tiene datos faltantes.

In [ ]:
X_california.info()

**Descripción de los atributos numéricos con `pd.DataFrame.describe()`**

In [ ]:
X_california.describe()

**Histograma de los atributos**

1. `MedInc` está expresado en una moneda que no parece ser dólares. Investigando sobre el data set vemos que "1" significa más o menos USD 10K de Median Income
2. `HouseAge` y `MedHouseVal` (la variable que quiero predecir) están cortadas en un valor máximo
3. Las dimensiones de los atributos tienen escalas muy distintas
4. Muchos atributos (`AveRooms`, `AveBedrms`, `Population`y `AveOccup`) tienen distribuciones muy sesgadas hacia la izquierda. Muchos valores bajos y pocos valores altos. 

In [ ]:
%matplotlib inline

X_california.hist(bins=50, figsize=(20,15)) 

In [ ]:
y_california.hist(bins=50)

## División en Entrenamiento y Evaluación

Dividimos aleatoriamente los datos en 80% para entrenamiento y 20% para evaluación:

In [ ]:
from sklearn.model_selection import train_test_split

X, y = california['data'], california['target']
X_train, X_test, y_train, y_test = train_test_split(X, y, train_size=0.8, random_state=42)
X_train.shape, X_test.shape

In [ ]:
y.shape

In [ ]:
y_test.shape

## Ejercicio 1: Descripción de los Datos y la Tarea

Responda las siguientes preguntas:

1. ¿De qué se trata el conjunto de datos?
2. ¿Cuál es la variable objetivo que hay que predecir? ¿Qué significado tiene?
3. ¿Qué información (atributos) hay disponibles para hacer la predicción?
4. ¿Qué atributos imagina ud. que serán los más determinantes para la predicción?
5. ¿Qué problemas observa a priori en el conjunto de datos? ¿Observa posibles sesgos, riesgos, dilemas éticos, etc? Piense que los datos pueden ser utilizados para hacer predicciones futuras.

**No hace falta escribir código para responder estas preguntas.**

## Ejercicio 2: Visualización de los Datos

1. Para cada atributo de entrada, haga una gráfica que muestre su relación con la variable objetivo.
2. Estudie las gráficas, identificando **a ojo** los atributos que a su criterio sean los más informativos para la predicción.
3. Para ud., ¿cuáles son esos atributos? Lístelos en orden de importancia.

In [ ]:
california['feature_names']

Ploteo la variable target en función de todas las features iterativamente.\
Uso `alpha=0.05` para añadir transparencia y ver mejor la densidad de puntos.

In [ ]:
for feature in california['feature_names']:
    selector = (np.array(california['feature_names']) == feature)
    plt.scatter(X[:,selector], y, alpha=0.1, label='datos')
    plt.title(feature)
    plt.show()

**2. Inspección de atributos**

El atributo más informativo parece ser el `MedInc` porque parece que la variable target cambia en función de esta variable que de las otras.\

**3. Atributos más informativos**

En órden de importancia `MedInc` y en menos medido `Latitude` y `Longitude`

## Ejercicio 3: Regresión Lineal

1. Seleccione **un solo atributo** que considere puede ser el más apropiado.
2. Instancie una regresión lineal de **scikit-learn**, y entrénela usando sólo el atributo seleccionado.
3. Evalúe, calculando error cuadrático medio para los conjuntos de entrenamiento y evaluación.
4. Grafique el modelo resultante, junto con los puntos de entrenamiento y evaluación.
5. Interprete el resultado, haciendo algún comentario sobre las cualidades del modelo obtenido.

**Observación:** Con algunos atributos se puede obtener un error en test menor a 50.

In [ ]:
# 1. Resolver acá. Ayuda:
feature = 'MedInc'
#selector = california['feature_names'].index(feature)
selector = (np.array(california['feature_names']) ==  feature)
X_train_f = X_train[:, selector]
X_test_f = X_test[:, selector]
X_train_f.shape, X_test_f.shape

In [ ]:
X_train_f[1]

In [ ]:
# 2. Instanciar y entrenar acá.
from sklearn.linear_model import LinearRegression

lin_reg = LinearRegression()
lin_reg.fit(X_train_f, y_train)

In [ ]:
lin_reg.coef_

Un RMSE de 8420 significa que en promedio mis prediciones en este modelo estuvieron 

In [ ]:
from sklearn.metrics import mean_squared_error

# predicciones para el set de entrenamiento
y_train_pred_f = lin_reg.predict(X_train_f)

# predicciones para el set de testeo
y_pred_f = lin_reg.predict(X_test_f)

train_rmse = np.sqrt(mean_squared_error(y_train, y_train_pred_f))
test_rmse = np.sqrt(mean_squared_error(y_test, y_pred_f))

print(f'RMSE de entrenamiento (USD): {train_rmse * 1e4}')
print(f'RMS de testeo (USD): {test_rmse * 1e4}')

In [ ]:
# 4. Graficar acá. Ayuda:
x_start = min(np.min(X_train_f), np.min(X_test_f))
x_end = max(np.max(X_train_f), np.max(X_test_f))
x = np.linspace(x_start, x_end, 200).reshape(-1, 1)
plt.plot(x, lin_reg.predict(x), color="tomato", label="modelo")

plt.scatter(X_train_f, y_train, facecolor="dodgerblue", edgecolor="k", label="train")
plt.scatter(X_test_f, y_test, facecolor="white", edgecolor="k", label="test")
plt.title(feature)
plt.legend()
plt.show()

**5. Interpretación**

El modelo no es muy bueno, con un RMSE de 6900 USD en entrenamiendo y 7000 dolares en testeo.\
Al ser similares los errores de entramiento y testeo me indica que no hay sobreajuste (overfitting).\
En promedio el modelo tiene un error de 7000 USD, lo cual no es muy bueno.

## Ejercicio 4: Regresión Polinomial

En este ejercicio deben entrenar regresiones polinomiales de diferente complejidad, siempre usando **scikit-learn**.

Deben usar **el mismo atributo** seleccionado para el ejercicio anterior.

1. Para varios grados de polinomio, haga lo siguiente:
    1. Instancie y entrene una regresión polinomial.
    2. Prediga y calcule error en entrenamiento y evaluación. Imprima los valores.
    3. Guarde los errores en una lista.
2. Grafique las curvas de error en términos del grado del polinomio.
3. Interprete la curva, identificando el punto en que comienza a haber sobreajuste, si lo hay.
4. Seleccione el modelo que mejor funcione, y grafique el modelo conjuntamente con los puntos.
5. Interprete el resultado, haciendo algún comentario sobre las cualidades del modelo obtenido.

**Observación:** Con algunos atributos se pueden obtener errores en test menores a 40 e incluso a 35.

1. Para varios grados de polinomio, haga lo siguiente:
    1. Instancie y entrene una regresión polinomial.
    2. Prediga y calcule error en entrenamiento y evaluación. Imprima los valores.
    3. Guarde los errores en una lista.

In [ ]:
X_train_f.shape

In [ ]:
from sklearn.preprocessing import PolynomialFeatures

# Crear las feautures polinomiales
pf = PolynomialFeatures(5)  # Instanciar el generador de features polinomiales
#pf.fit(X_train_f)  # Crear las features polinomiales

X_train_f_poly = pf.fit_transform(X_train_f)
X_train_f_poly.shape

In [ ]:
pf = PolynomialFeatures(1)
lr = LinearRegression(fit_intercept=False)  # el bias term ya está como feature por polynomial features

pf.fit_transform(X_train_f)

In [ ]:
X_train_f

In [ ]:
from sklearn.pipeline import make_pipeline

model = make_pipeline(pf, lr)
model.fit(X_train_f, y_train)

In [ ]:
from sklearn.linear_model import LinearRegression
from sklearn.preprocessing import PolynomialFeatures
from sklearn.pipeline import make_pipeline
from sklearn.metrics import mean_squared_error

train_rmse_list = []
test_rmse_list = []

degrees = [0, 1, 2, 3, 4, 5, 6, 7, 8, 9]

for degree in degrees:
    
    pf = PolynomialFeatures(degree)
    lr = LinearRegression(fit_intercept=False)  # el bias term ya está como feature por polynomial features

    model = make_pipeline(pf, lr)
    model.fit(X_train_f, y_train)
    
    print(f"Coeficientes del modelo de polinomio {degree}: {model['linearregression'].coef_}")

    y_train_f_pred = model.predict(X_train_f)
    y_test_f_pred = model.predict(X_test_f)
    
    train_rmse = np.sqrt(mean_squared_error(y_train, y_train_f_pred)) * 1e4
    test_rmse = np.sqrt(mean_squared_error(y_test, y_test_f_pred)) * 1e4
    
    print(f'Train RMSE: {train_rmse}')
    print(f'Test RMSE: {test_rmse}')

    train_rmse_list.append(train_rmse)
    test_rmse_list.append(test_rmse)
    
    x_start = min(np.min(X_train_f), np.min(X_test_f))
    x_end = max(np.max(X_train_f), np.max(X_test_f))
    x = np.linspace(x_start, x_end, 200).reshape(-1, 1)
    plt.plot(x, model.predict(x), color="red", label="modelo")

    plt.scatter(X_train_f, y_train, facecolor="dodgerblue", label="train", alpha=0.3, edgecolor="k")
    plt.scatter(X_test_f, y_test, facecolor="yellow", label="test", alpha=0.3, edgecolor="k")
    plt.title(f'{feature} poly {degree}')
    plt.legend()
    plt.show()





In [ ]:
train_rmse, test_rmse

In [ ]:
# 2. Graficar curvas de error acá.
plt.plot(degrees, train_rmse_list, color="blue", label="train")
plt.plot(degrees, test_rmse_list, color="red", label="test")
plt.legend()
plt.xlabel("degree")
plt.ylabel("error")
plt.show()

3. Interprete la curva, identificando el punto en que comienza a haber sobreajuste, si lo hay.

4. Seleccione el modelo que mejor funcione, y grafique el modelo conjuntamente con los puntos.



5. Interprete el resultado, haciendo algún comentario sobre las cualidades del modelo obtenido.

## Ejercicio 5: Regresión con más de un Atributo

En este ejercicio deben entrenar regresiones que toman más de un atributo de entrada.

1. Seleccione **dos o tres atributos** entre los más relevantes encontrados en el ejercicio 2.
2. Repita el ejercicio anterior, pero usando los atributos seleccionados. No hace falta graficar el modelo final.
3. Interprete el resultado y compare con los ejercicios anteriores. ¿Se obtuvieron mejores modelos? ¿Porqué?

In [ ]:
# 1. Resolver acá. Ayuda (con dos atributos):
selector = (np.array(california['feature_names']) == 'HouseAge') | (np.array(california['feature_names']) == 'AveRooms')

X_train_fs = X_train[:, selector]
X_test_fs = X_test[:, selector]
X_train_fs.shape, X_test_fs.shape

In [ ]:
# 2. Resolver acá.

from sklearn.linear_model import LinearRegression
from sklearn.preprocessing import PolynomialFeatures
from sklearn.pipeline import make_pipeline
from sklearn.metrics import mean_squared_error

train_rmse_list = []
test_rmse_list = []

degrees = [0, 1, 2, 3, 4]

for degree in degrees:
    
    pf = PolynomialFeatures(degree)
    lr = LinearRegression(fit_intercept=False)  # el bias term ya está como feature por polynomial features

    model = make_pipeline(pf, lr)
    model.fit(X_train_fs, y_train)
    
    print(f"Coeficientes del modelo de polinomio {degree}: {model['linearregression'].coef_}")

    y_train_fs_pred = model.predict(X_train_fs)
    y_test_fs_pred = model.predict(X_test_fs)
    
    train_rmse = np.sqrt(mean_squared_error(y_train, y_train_fs_pred)) * 1e4
    test_rmse = np.sqrt(mean_squared_error(y_test, y_test_fs_pred)) * 1e4
    
    print(f'Train RMSE: {train_rmse}')
    print(f'Test RMSE: {test_rmse}')

    train_rmse_list.append(train_rmse)
    test_rmse_list.append(test_rmse)
    

In [ ]:
# 2. Graficar curvas de error acá.
plt.plot(degrees, train_rmse_list, color="blue", label="train")
plt.plot(degrees, test_rmse_list, color="red", label="test")
plt.legend()
plt.xlabel("degree")
plt.ylabel("error")
plt.show()

**3. Responder acá.**

Cuando usamos dos variables o muchas más (con sus versiones polinómicas) el modelo sobreajusta rápidamente luego de ser entrenado con las features de polinomio tres. Lo podemos ver porque la RMSE de entrenamiento es baja mientras que la RMSE de test es cada vez más alta.

De hecho ningún modelo de esta versión con más de una variable logra RMSE mejores que el mejor modelo de una variable. 

## Más ejercicios (opcionales)

### Ejercicio 6: A Todo Feature

Entrene y evalúe regresiones pero utilizando todos los atributos de entrada (va a andar mucho más lento). Estudie los resultados.

### Ejercicio 7: Regularización

Entrene y evalúe regresiones con regularización "ridge". Deberá probar distintos valores de "alpha" (fuerza de la regularización). ¿Mejoran los resultados?
